# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from pprint import pprint


In [ ]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [ ]:
products_by_client = data.groupby(['CustomerID', 'ProductID']).sum()

#Remove 'SalesID'
products_by_client = products_by_client.drop(columns=['SalesID'])
products_by_client.head()

In [ ]:
#You will need to group by CustomerID and ProductName and then sum the Quantity field.


df=data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

df.head()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [ ]:
pivot_table = pd.pivot_table(data, values = 'Quantity', index = 'ProductName', columns = 'CustomerID')
pivot_table = pivot_table.fillna(0)
pivot_table.head()

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [ ]:
pivot_table_transpose = pivot_table.transpose()
customer_similarity_matrix = pd.DataFrame(squareform(pdist(pivot_table_transpose, 'euclidean')),
                                          columns = pivot_table_transpose.index, index = pivot_table_transpose.index)

customer_similarity_matrix.head()


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [ ]:
list_top5 = list(customer_similarity_matrix[33].sort_values(ascending=False).index)[:5]

In [ ]:
list_top5

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [ ]:
records = df[df['CustomerID'].isin(list_top5)]
records.head()

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
records_agg = records.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
records_agg.head()


In [ ]:
records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
records_agg.head()



## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

#### Merge the ranked products data frame with the customer product matrix on the ProductName field.

In [ ]:
products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
products_similar_customers.head()

#### Filter for records where the chosen customer has not purchased the product.


In [ ]:
not_purchased = pivot_table[pivot_table[33]==0]
not_purchased.head()

#### Show the top 5 results.

In [ ]:
products_not_purchased = not_purchased.merge(products_similar_customers, 
    left_on= 'ProductName', right_on= 'ProductName').sort_values('Quantity', ascending= False)
products = products_not_purchased[1:6].index.to_list()
products

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

#### Create an empty dictionary that will hold the recommendations for all customers.

In [ ]:
recommendations = {}

#### Create a list of unique CustomerIDs to iterate over

In [ ]:
customers_ids = data['CustomerID'].unique()
customers_ids_list = customers_ids.tolist()
customers_ids_list

#### Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created

In [ ]:
recommendations = {}
customers_ind = data.index.tolist()
#customers_ids_list = customers_ids.tolist()
#customers_ids_list

for customers in customers_ind:
    list_top5 = list(customer_similarity_matrix[33].sort_values(ascending=False).index)[:5]
    records = df[df['CustomerID'].isin(list_top5)]
    records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
    products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
    not_purchased = pivot_table[pivot_table[33]==0]
    products_not_purchased = not_purchased.merge(products_similar_customers, 
    left_on= 'ProductName', right_on= 'ProductName').sort_values('Quantity', ascending= False)
    products = products_not_purchased[1:6].index.to_list()
    recommendations.update({customers:products})
        

In [ ]:
customers_ids = [data['CustomerID'].unique()]
customers_ids

In [ ]:
for customers in customers_ids:
    for customer in customers:
        list_top5 = list(customer_similarity_matrix[customer].sort_values(ascending=False).index)[:5]
        records = df[df['CustomerID'].isin(list_top5)]
        records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
        products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
        not_purchased = pivot_table[pivot_table[33]==0]
        products_not_purchased = not_purchased.merge(products_similar_customers, 
    left_on= 'ProductName', right_on= 'ProductName').sort_values('Quantity', ascending= False)
        recommendations.update({customers_ids_list:products_not_purchased})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
results_df = pd.DataFrame(recommendations)
results_df.head()

In [ ]:
# Hago df desde diccionario y reinicio index:
results = pd.DataFrame.from_dict(recommendations, columns = ['CustomerID', 'Product1', 'Product2', 'Product3', 'Product4','Product5'], orient = 'index').reset_index()

# Cambio el nombre del index:
#results.index.names = ['CustomerID']
results.head()


In [ ]:
results2 = pd.DataFrame.from_dict(recommendations_for_everyone, orient='index').reset_index()
results2.columns = ['CustomerID','Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results2.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.